In [ ]:
import cv2
import torch
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

# Load YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

def detect_vehicles(image):
    results = model(image)
    detections = results.pandas().xyxy[0]  # Extracting detections as a pandas DataFrame

    vehicles = []
    for _, row in detections.iterrows():
        if row['name'] in ["car", "bus", "truck", "motorcycle", "bicycle"]:
            x, y, w, h = int(row['xmin']), int(row['ymin']), int(row['xmax']) - int(row['xmin']), int(row['ymax']) - int(row['ymin'])
            vehicles.append((x, y, w, h, row['name']))

    return vehicles

def draw_bounding_boxes(image, vehicles):
    for (x, y, w, h, label) in vehicles:
        # Draw the bounding box
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)

        # Add the label and measurements
        text = f"{label}: {w}x{h}px"
        cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return image

# Load the image
image_path = "/content/sample_data/0022713.jpeg"  # Replace with the path to your image
image = cv2.imread(image_path)

# Detect vehicles in the image
vehicles = detect_vehicles(image)

# Draw bounding boxes around the detected vehicles and display their measurements
image_with_boxes = draw_bounding_boxes(image, vehicles)

# Convert the image from BGR to RGB (for displaying with matplotlib)
image_with_boxes_rgb = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)

# Display the image with bounding boxes
plt.figure(figsize=(10, 10))
plt.imshow(image_with_boxes_rgb)
plt.axis('off')
plt.show()

# Save the image with bounding boxes
output_path = "/content/Screenshot 2024-07-04 143632.png"  # Replace with the path to save the output image
cv2.imwrite(output_path, image_with_boxes)

# Save annotations to a CSV file
annotations = []
for (x, y, w, h, label) in vehicles:
    annotations.append([x, y, x + w, y + h, label])

annotations_df = pd.DataFrame(annotations, columns=['xmin', 'ymin', 'xmax', 'ymax', 'label'])
annotations_path = "/content/sample_data/daily_weather.csv"  # Replace with the path to save the annotations
annotations_df.to_csv(annotations_path, index=False)
